In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import fnmatch
import errno    
import glob
from pysradb import SRAdb

/home/saket/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/saket/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
db = SRAdb('/home/saket/SRAmetadb.sqlite')


# List tables

In [3]:
sra_tables = db.list_tables()
sra_tables

['metaInfo',
 'submission',
 'study',
 'sample',
 'experiment',
 'run',
 'sra',
 'sra_ft',
 'sra_ft_content',
 'sra_ft_segments',
 'sra_ft_segdir',
 'col_desc',
 'fastq']

# List fields in a table

In [4]:
db.list_fields('study')

['study_ID',
 'study_alias',
 'study_accession',
 'study_title',
 'study_type',
 'study_abstract',
 'broker_name',
 'center_name',
 'center_project_name',
 'study_description',
 'related_studies',
 'primary_study',
 'sra_link',
 'study_url_link',
 'xref_link',
 'study_entrez_link',
 'ddbj_link',
 'ena_link',
 'study_attribute',
 'submission_accession',
 'sradb_updated']

# Describe columns in a table

In [5]:
db.desc_table('study')

,cid,name,dtype,notnull,dflt_value,pk
0,0,study_ID,REAL,0,None,0
1,1,study_alias,TEXT,0,None,0
2,2,study_accession,TEXT,0,None,0
3,3,study_title,TEXT,0,None,0
4,4,study_type,TEXT,0,None,0
5,5,study_abstract,TEXT,0,None,0
6,6,broker_name,TEXT,0,None,0
7,7,center_name,TEXT,0,None,0
8,8,center_project_name,TEXT,0,None,0
9,9,study_description,TEXT,0,None,0


In [6]:
sorted(db.desc_table('sra_ft').name.tolist())

['SRR_bamFile',
 'SRX_bamFile',
 'SRX_fastqFTP',
 'adapter_spec',
 'anonymized_name',
 'base_caller',
 'bases',
 'center_project_name',
 'common_name',
 'description',
 'design_description',
 'experiment_ID',
 'experiment_accession',
 'experiment_alias',
 'experiment_attribute',
 'experiment_entrez_link',
 'experiment_name',
 'experiment_title',
 'experiment_url_link',
 'individual_name',
 'instrument_model',
 'instrument_name',
 'library_construction_protocol',
 'library_layout',
 'library_name',
 'library_selection',
 'library_source',
 'library_strategy',
 'multiplier',
 'number_of_levels',
 'platform',
 'platform_parameters',
 'primary_study',
 'qtype',
 'quality_scorer',
 'read_spec',
 'related_studies',
 'run_ID',
 'run_accession',
 'run_alias',
 'run_attribute',
 'run_center',
 'run_date',
 'run_entrez_link',
 'run_url_link',
 'sample_ID',
 'sample_accession',
 'sample_alias',
 'sample_attribute',
 'sample_entrez_link',
 'sample_name',
 'sample_url_link',
 'sequence_space',
 'sp

# Query

In [7]:
db.get_query('SELECT * FROM study LIMIT 3')

,broker_name,center_name,center_project_name,ddbj_link,ena_link,primary_study,related_studies,sra_link,sradb_updated,study_ID,...,study_accession,study_alias,study_attribute,study_description,study_entrez_link,study_title,study_type,study_url_link,submission_accession,xref_link
0,None,KEIO,Bacillus subtilis subsp. natto BEST195,None,None,None,None,None,2016-11-20 16:42:15,1.0,...,DRP000001,DRP000001,None,None,None,Bacillus subtilis subsp. natto BEST195 genome ...,Whole Genome Sequencing,None,DRA000001,pubmed: 20398357 || pubmed: 25329997
1,None,KEIO,Bacillus subtilis subsp. subtilis str. 168,None,None,None,None,None,2017-09-21 20:23:35,2.0,...,DRP000002,DRP000002,None,Whole genome resequencing of B. subtilis subti...,None,Model organism for prokaryotic cell differenti...,Whole Genome Sequencing,None,DRA000002,pubmed: 20398357
2,None,UT-MGS,Integrative Transcriptome Analysis,None,None,None,None,None,2016-11-20 16:42:16,3.0,...,DRP000003,DRP000003,None,Although recent studies have revealed that the...,None,Comprehensive identification and characterizat...,Transcriptome Analysis,DBTSS: http://dbtss.hgc.jp/,DRA000003,pubmed: 20400770


# Number of rows in each table

In [8]:
db.get_table_counts()

,count
metaInfo,2
submission,1017441
study,161391
sample,4439622
experiment,5011580
run,5667504
sra,5568559
sra_ft,5568559
sra_ft_content,5568559
sra_ft_segments,3007295


# Which study types are the most prominent?

In [9]:
db.get_query('SELECT study_type AS StudyType, count(*) AS Number FROM "study" GROUP BY study_type order by Number DESC')

,Number,StudyType
0,45968,Whole Genome Sequencing
1,44591,None
2,38516,Other
3,16149,Transcriptome Analysis
4,14415,Metagenomics
5,763,Population Genomics
6,607,Epigenetics
7,230,Exome Sequencing
8,110,Cancer Genomics
9,31,Pooled Clone Sequencing


# Which Sequencing Instruments are most common?

In [10]:
instruments = db.get_query('SELECT instrument_model AS "Instrument Model", count( * ) AS Experiments FROM "experiment" GROUP BY instrument_model order by Experiments DESC').sort_values(by=['Instrument Model'])
instruments

,Experiments,Instrument Model
21,8704,454 GS
32,986,454 GS 20
9,49413,454 GS FLX
50,10,454 GS FLX
4,148483,454 GS FLX Titanium
15,16702,454 GS FLX+
13,23956,454 GS Junior
53,1,AB 310 Genetic Analyzer
42,111,AB 3130 Genetic Analyzer
44,55,AB 3130xL Genetic Analyzer


# Which assays are most common?

In [11]:
assays = db.get_query('SELECT library_strategy AS "Library Strategy", count( * ) AS Runs FROM "experiment" GROUP BY library_strategy order by Runs DESC')
assays

,Library Strategy,Runs
0,None,1665711
1,WGS,1092231
2,AMPLICON,690441
3,RNA-Seq,627662
4,OTHER,314145
5,WXS,237358
6,CLONE,89555
7,ChIP-Seq,82832
8,POOLCLONE,53738
9,Bisulfite-Seq,32335


# List SRA Run table

In [12]:
df = db.sra_convert('SRP017942')
df

,study_accession,experiment_accession,experiment_title,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP017942,SRX217028,GSM1063575: 293T_GFP; Homo sapiens; RNA-Seq,SRR648667,9606,other,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1806641316,50184481,None,36.0
1,SRP017942,SRX217029,GSM1063576: 293T_GFP_2hrs_severe_Heat_Shock; H...,SRR648668,9606,other,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,3436984836,95471801,None,36.0
2,SRP017942,SRX217030,GSM1063577: 293T_Hspa1a; Homo sapiens; RNA-Seq,SRR648669,9606,other,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,3330909216,92525256,None,36.0
3,SRP017942,SRX217031,GSM1063578: 293T_Hspa1a_2hrs_severe_Heat_Shock...,SRR648670,9606,other,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,3622123512,100614542,None,36.0
4,SRP017942,SRX217956,GSM794854: 3T3-Control-Riboseq; Mus musculus; ...,SRR649752,10090,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,594945396,16526261,None,36.0
5,SRP017942,SRX217957,GSM794855: 3T3-8hrs-mild-HS-Riboseq; Mus muscu...,SRR649753,10090,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,730843272,20301202,None,36.0
6,SRP017942,SRX217958,GSM794856: 3T3-2hrs-severe-HS-Riboseq; Mus mus...,SRR649754,10090,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1785538908,49598303,None,36.0
7,SRP017942,SRX217027,GSM1063574: 3T3-Hsp70_inhibitor_3hr; Mus muscu...,SRR648666,10090,other,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,5932552920,148313823,None,40.0
8,SRP017942,SRX217959,GSM794857: 3T3-Control-Riboseq-rep2; Mus muscu...,SRR649755,10090,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,3713863640,92846591,None,40.0
9,SRP017942,SRX217960,GSM794858: 3T3-8hrs-mild-HS-Riboseq-rep2; Mus ...,SRR649756,10090,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,3967599960,99189999,None,40.0


In [13]:
df = db.sra_convert('SRP002605')
df

,study_accession,experiment_accession,experiment_title,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP002605,SRX021966,GSM546920: Footprint_mock_32hr_runs1-2,SRR057511,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546920: Footprint_mock_32hr_runs1-2,173069316,4807481,None,36.0
1,SRP002605,SRX021966,GSM546920: Footprint_mock_32hr_runs1-2,SRR057512,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546920: Footprint_mock_32hr_runs1-2,670274352,18618732,None,36.0
2,SRP002605,SRX021967,GSM546921: mRNASeq_mock_32hr_runs1-3,SRR057513,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546921: mRNASeq_mock_32hr_runs1-3,159276276,4424341,None,36.0
3,SRP002605,SRX021967,GSM546921: mRNASeq_mock_32hr_runs1-3,SRR057515,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546921: mRNASeq_mock_32hr_runs1-3,92929104,2581364,None,36.0
4,SRP002605,SRX021968,GSM546922: Footprint_miR-155_32hr_runs1-2,SRR057516,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546922: Footprint_miR-155_32hr_runs1-2,212446440,5901290,None,36.0
5,SRP002605,SRX021968,GSM546922: Footprint_miR-155_32hr_runs1-2,SRR057517,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546922: Footprint_miR-155_32hr_runs1-2,522725688,14520158,None,36.0
6,SRP002605,SRX021969,GSM546923: mRNASeq_miR-155_32hr_runs1-3,SRR057518,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546923: mRNASeq_miR-155_32hr_runs1-3,216374004,6010389,None,36.0
7,SRP002605,SRX021969,GSM546923: mRNASeq_miR-155_32hr_runs1-3,SRR057519,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546923: mRNASeq_miR-155_32hr_runs1-3,207243972,5756777,None,36.0
8,SRP002605,SRX021969,GSM546923: mRNASeq_miR-155_32hr_runs1-3,SRR057520,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546923: mRNASeq_miR-155_32hr_runs1-3,94523436,2625651,None,36.0
9,SRP002605,SRX021970,GSM546924: Footprint_miR-1_32hr_runs1-2,SRR057521,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546924: Footprint_miR-1_32hr_runs1-2,212789088,5910808,None,36.0


In [14]:
df = db.sra_convert('SRP098789')
db.download(df=df, ascp_bin='/usr/bin/ascp')

b''
b'Session Stop  (Error: Client unable to connect to server (check UDP port and firewall))'


KeyboardInterrupt: 